In [41]:
from PyQt5.QtCore import QSettings
from serial import Serial
from time import sleep
import pyTMCL
from pyTMCL import Motor
from serial.tools.list_ports import comports
from PyQt5.QtCore import QSettings

In [42]:
class abstractMotor(object):
    def __init__(self, port, module_address, motor_id, confSettings):
        
        self.port=port
        self.module_address=module_address
        self.motor_id=motor_id
        self.confSettings=confSettings
        self.mottype='generic'
        
        # Connect, and get motor : returns Bus and Motor instances from pyTMCL module
        self.serial_port = Serial(self.port)
        self.bus = pyTMCL.connect(self.serial_port)
        self.motor = self.bus.get_motor(self.module_address, self.motor_id)
            
        try :
            self.confMot=QSettings(confSettings, QSettings.IniFormat) # motor configuration  files  
            self.vMax=int(self.confMot.value(self.mottype+'/vMax'))#1142
            self.iMax=int(self.confMot.value(self.mottype+'/iMax'))
            self.iStby=int(self.confMot.value(self.mottype+'/iStby'))
            self.accMax=int(self.confMot.value(self.mottype+'/accMax'))
            self.ustepRes=int(self.confMot.value(self.mottype+'/ustepRes')) # step resolution (0-8)
            
        except :
            self.serial_port.close()
            print('Failed to load config file')
        
        try : 
            self.motor.set_axis_parameter(4, self.iMax)   # max current frac
            self.motor.set_axis_parameter(5, self.accMax) # max acc
            self.motor.set_axis_parameter(140, self.ustepRes)  # µstep res
            self.motor.set_axis_parameter(4, self.vMax) # max vel
            self.motor.set_axis_parameter(7, self.iStby) # max standby current 
        except :
            self.serial_port.close()
            print('Failed to set motor config')
        
    def close(self):
        self.motor.stop()
        self.serial_port.close()
    
    def STOP(self):
        self.motor.stop()
        
    def callback(self):
        print ('''Position reached: %d steps'''%self.motor.get_axis_parameter(1))
            
    def move_absolute(self, steps):
        self.motor.move_absolute(steps, self.callback)
        
    def move_relative(self, steps):
        self.motor.move_relative(steps)
    
    def get_pos(self):
        return self.motor.get_axis_parameter(1)

            
            

In [43]:
FOS_motor = abstractMotor('COM5', 1, 0, 'configMoteur.ini')

In [44]:
old_pos = FOS_motor.get_pos()
print (old_pos)

FOS_motor.move_absolute(old_pos - 100000)

-102173


In [45]:
FOS_motor.STOP()

Position reached: -109441 steps


In [47]:
FOS_motor.close()



SerialException: Attempting to use a port that is not open

In [27]:
ports = comports()

In [34]:
for port in ports :
    print( port.device)


COM3
COM1
COM5
